# Processamento dos dados do sensor de pressao RBR
na laje do sheraton no periodo de novembro/2016

AtmosMarine

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as pl
from waveproc import WaveProc
%matplotlib qt

In [17]:
pathname = os.environ['HOME'] + '/Dropbox/atmosmarine/data/lajesheraton/Swell_reserva_201611/'

filename = 'Swell_reserva_201611_data.txt'

dd1 = pd.read_table(pathname + filename, sep=',', header=0, names=['date', 'pr', 'spr', 'dp'],
                   index_col='date', parse_dates=True)

#periodo de funcionamento do sensor
dd = dd1['2016-10-28 21:00':'2016-10-29 23:00']
dd2 = dd1['2016-11-05 21:00':'2016-11-06 23:00']

In [190]:
w = WaveProc(pathname)

d = []
sp = []
for i in np.arange(0,len(dd),2400):
        
    w.n1 = dd.pr.values[i:i+2400] - dd.pr.values[i:i+2400].mean()
    w.n2 = w.n1
    w.n3 = w.n1
    w.h = w.n1.mean()
    w.t = np.arange(0,len(w.n1)/2,0.5)
    w.dt = w.t[3] - w.t[2]
    w.fs = 2
    w.nfft = int(len(w.n1) / 2)
    
    w.timedomain()
    w.freqdomain()
    
    d.append({'date'   : dd.index[i],
              'hs'     : w.hs,
              'h10'    : w.h10,
              'hmax'   : w.hmax,
              'tmed'   : w.tmed,
              'thmax'  : w.thmax,
              'hm0'    : w.hm0,
              'tp'     : w.tp,
              'dp'     : w.dp,
              'tzamax' : w.tzamax
            })
    
    sp.append(w.sn1[:,1])

df = pd.DataFrame(d)
df = df.set_index('date')

sp = np.array(sp).T

In [5]:
pl.close('all')
ax = pl.contourf(df.index, w.sn1[:,0], sp, 100)
pl.ylim(w.sn1[0,0],0.1)
pl.ylabel('Freq. (Hz)')
cbar = pl.colorbar()
cbar.ax.set_ylabel('m2/Hz', rotation=270)

NameError: name 'df' is not defined

In [22]:
pl.hist(dd.pr-dd.pr.mean(), 50)
pl.hist(dd2.pr-dd2.pr.mean(), 50)
pl.xlim(-4,4)
pl.grid()

In [20]:
dd.shape, dd2.shape

((187320, 3), (187320, 3))

In [171]:
pl.plot(w.sn1[:,0], w.sn1[:,1])

In [145]:
pl.close('all')
pl.plot(df.index, df.hm0, df.index, df.hs)
pl.legend(['hm0', 'hs'], loc=2)
pl.ylabel('Metros')
pl.grid()
pl.twinx()
pl.plot(df.index, df.tp,'r.')
pl.legend(['tp'], loc=1)
pl.ylabel('Segundos')